# Latent Factor Models for Collaborative Filtering

Load Pandas, we are going to need it for manipulating data

In [1]:
import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

In [2]:
data = pd.read_csv("jester-data-1.csv",header=None)
d = data.to_latex()

In [3]:
crop_data=data.copy()
crop_data=crop_data.values[0:1979,1:]
print(crop_data.shape[0],crop_data.shape[1])

1979 100


Now load the data

In [4]:

#text_file = open("Output.txt", "w")
#text_file.write(d)
#text_file.close()

crop_train=crop_data.copy()
size=round(0.1*crop_data.shape[0]*crop_data.shape[1])
#print(size)
randx=np.random.randint(0,crop_data.shape[1]-1,size)
randy=np.random.randint(0,crop_data.shape[0]-1,size)

print(size, max(randx), max(randy))


19790 98 1977


In [5]:
for i in range(size):
    crop_train[randy[i],randx[i]] = 99



In [ ]:
type(crop_train)

In [ ]:
np.isnan(12)


In [6]:
#99 in data
A=0
for i in crop_data:
    for j in i:
        if (j==99):
            A+=1
print(A)

57824


In [7]:
#99 in data_test
B=0
for u in crop_train:
    for v in u:
        if (v==99):
            B+=1
print(B)
#print(862910.0/2523283 )

71308


In [ ]:
def is99(number):
    if number == 99:
        return True
    else:
        return False

In [ ]:
print(is99(99.0))

In [8]:
n_features = 2

user_ratings = crop_train
user_ratings_test=crop_data
latent_user_preferences = np.random.random((user_ratings.shape[0], n_features))
latent_item_features = np.random.random((user_ratings.shape[1],n_features))

In [9]:
latent_item_features

array([[0.212, 0.82 ],
       [0.489, 0.75 ],
       [0.43 , 0.626],
       [0.675, 0.265],
       [0.25 , 0.573],
       [0.104, 0.284],
       [0.672, 0.542],
       [0.637, 0.868],
       [0.126, 0.91 ],
       [0.385, 0.059],
       [0.447, 0.675],
       [0.626, 0.025],
       [0.506, 0.814],
       [0.061, 0.619],
       [0.608, 0.687],
       [0.338, 0.577],
       [0.093, 0.33 ],
       [0.513, 0.002],
       [0.699, 0.935],
       [0.999, 0.195],
       [0.222, 0.152],
       [0.835, 0.191],
       [0.577, 0.964],
       [0.569, 0.862],
       [0.342, 0.505],
       [0.074, 0.703],
       [0.84 , 0.515],
       [0.532, 0.224],
       [0.211, 0.27 ],
       [0.797, 0.655],
       [0.76 , 0.248],
       [0.351, 0.01 ],
       [0.469, 0.595],
       [0.999, 0.738],
       [0.998, 0.738],
       [0.661, 0.874],
       [0.886, 0.138],
       [0.766, 0.193],
       [0.087, 0.666],
       [0.067, 0.384],
       [0.786, 0.019],
       [0.164, 0.168],
       [0.013, 0.941],
       [0.6

In [10]:
latent_user_preferences

array([[0.948, 0.018],
       [0.081, 0.438],
       [0.917, 0.728],
       ...,
       [0.519, 0.86 ],
       [0.687, 0.384],
       [0.008, 0.681]])

In [16]:
def predict_rating(user_id,item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)

def train(user_id, item_id, rating,alpha = 0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  ( prediction_rating- rating );
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err

def traintest(user_id, item_id, rating):
    prediction_rating = predict_rating(user_id, item_id)
    err =  ( prediction_rating- rating );
    return err

def is99(number):
    if number == 99:
        return True
    else:
        return False

def sgd(iterations = 300):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(0,iterations):
        error = []
        error_test=[]
        for user_id in range(0,latent_user_preferences.shape[0]):
            for item_id in range(0,latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                rating_test=user_ratings_test[user_id][item_id]
                if(not is99(rating)):
                    err = train(user_id,item_id,rating)
                    err_test=traintest(user_id,item_id,rating_test)
                    error.append(err)
                    error_test.append(err_test)
        mse = (np.array(error) ** 2).mean()
        mse_test = (np.array(error_test) ** 2).mean()
        if(iteration%10 == 0 ):
            print(iteration,mse,mse_test)
                    
                    
    


In [17]:
sgd()


0 18.862145950535066 18.82496542727898
10 18.14354679388944 18.102153721107893
20 17.572841486956886 17.527363623338683
30 17.257676520453558 17.20905694637217
40 17.105235665957835 17.05460302898668
50 17.02625343509197 16.974417886483852
60 16.98085586517978 16.928267384978078
70 16.95284891284349 16.899741659715225
80 16.934783800804652 16.881286720291254
90 16.922767582447253 16.86895982411717
100 16.914588341667834 16.86052385564169
110 16.908918956638836 16.854637399716793
120 16.90493138952703 16.850463310857588
130 16.90209309374547 16.847462692820965


KeyboardInterrupt: 